# はじめに

このチュートリアルでは**グラフの集合**に対するさまざまな計算を高速に実行するためのPythonライブラリである[Graphillion](https://github.com/takemaru/graphillion)を紹介します．Graphillionを用いることでグラフに対する以下のような計算を，数行のコードで簡単に実行できます．

- グラフ中の2つの頂点を結ぶ経路のうち，特定の頂点を経由するものの個数を調べる
- すべての頂点をたどる巡回路のうち，辺の重みの和が最長のものを求める．
- 通信ネットワークが故障せずに動作する確率を計算する．

これらの計算はすべて**NP困難**とよばれる，計算が難しい問題の一種であることが知られています．NP困難な問題を解くために必要な計算時間は問題サイズ対して指数的に増加します．そのため，問題サイズが大きくなると，現代の計算機では計算に年単位の時間がかかってしまうことも容易に起こりえます．Graphillionはこうした難しい計算を現実的な時間で解く機能を提供します．

Graphillionの内部では**ゼロサプレス型二分決定グラフ (ZDD)** とよばれるデータ構造と，ZDDを用いた最先端のアルゴリズム技術が用いられています．このチュートリアルではGraphillionのなかでZDDがどのように扱われているかについても簡単に解説します．なお，GraphillionはZDDの存在を完全に隠蔽しているので，ユーザはZDDに関する知識がなくてもグラフの集合を直観的に扱うことができます．


## このチュートリアルで学べること

このチュートリアルを通じて以下を取得してもらうことを目標としています．

- Graphillionの基本的な利用方法．
- Graphillionが特に効果を発揮するユースケース．
- Grapillionの動作原理．特にZDDが果たす役割について．

## Graphillionの位置づけ

グラフを扱うためのライブラリは多数存在します．たとえば [NetworkX](https://networkx.github.io/) はグラフをを扱うな代表的なPythonライブラリであり，グラフを対象とした様々なアルゴリズムの実行や可視化などの豊富な機能を有しています．

Graphillionはグラフの集合の処理に特化した機能を提供するライブラリであり，NetworkXのような多様な機能は有していません．一方でグラフ集合の処理に関しては他のソフトウェアでは実現できない多数の機能を有しています．グラフ集合を扱うことで実現できることについては以降のチュートリアルで紹介します．

なお，GraphillionはNetworkXと連携して利用することも可能です．たとえばGraphillionの実行結果をNetworkXに渡して，NetworkXの各種機能を利用することができます．

## Google Colaboratoryの利用
本チュートリアルで用いるプログラムはすべて[Google Colaboratory](https://colab.research.google.com/notebooks/intro.ipynb) （Colab）上で動かすことができます．Calabを用いることでPythonの実行環境を用意することなくGraphillionの機能を試すことができます．

Colabを利用するためにはGoogleアカウントが必要となります．Colabはほとんどの主要なブラウザで動作します．最新バージョンのChrome, Firefox, Safariでは完全に動作することが検証されています．


## Colab環境のセットアップ
チュートリアルを実行する前に，以下のセルを実行してColab環境にGraphillionとチュートリアル用のpythonモジュールをインストールします．

左上の実行ボタンをクリックするか，セルを選択して`ctrl+enter`でセルを実行することができます．

In [1]:
!pip install graphillion
!git clone https://github.com/nsnmsak/graphillion_tutorial
!cp graphillion_tutorial/ja/tutorial_util.py .

     |████████████████████████████████| 1.1MB 4.3MB/s 
  Created wheel for graphillion: filename=Graphillion-1.3-cp36-cp36m-linux_x86_64.whl size=1728298 sha256=b546e68c230c25e659e7e5f895dfe49aa5c5bf1aebb280af565ea4277a3c250a
  Stored in directory: /root/.cache/pip/wheels/f5/11/37/c871e165ed219369e237be48be2c7cf989bc034699aeee77c8
Successfully built graphillion
Cloning into 'graphillion_tutorial'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 112 (delta 50), reused 82 (delta 27), pack-reused 0
Receiving objects: 100% (112/112), 6.33 MiB | 15.22 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [0]:
from graphillion import GraphSet, tutorial
from tutorial_util import draw_zdd, draw_subgraph

GraphSet.set_universe(tutorial.grid(2, 2))
paths = GraphSet.paths(1, 9)



## アウトライン

2. [グラフ入門](02_graph_theory.ipynb):  本チュートリアルを理解するために必要となるグラフ理論の基礎について説明します．グラフとは何か，なぜグラフ理論が有益なのかを簡単に説明します．既にグラフ理論に習熟されている方は4章に進んでください．

3. [グラフと組合せ爆発](03_graph_and_combinatorial_explosion.ipynb): グラフの問題を解く際，しばしば指数的に存在する部分グラフを扱わなければならない場面がでてきます．この章ではそのような組合せ爆発について解説します．

4. [Graphillionに触れてみよう](04_graphillion_first_step.ipynb): いよいよGraphillionの説明に入ります．ま←あ有名な「数え上げお姉さん問題」を題材に，Graphillionを用いたグラフ集合処理の例を紹介します．

5. [Graphillionの内部](05_graphillion_and_zdd.ipynb): Graphillionの内部ではZDDとよばれるデータ構造が利用されています．GraphillionはZDDを意識せずとも利用することができますが，Graphillionの挙動を理解するためにはZDDの理解が不可欠です．この章ではGraphillionにおいてZDDが果たす役割について説明します．

6. [GraphSet](06_graph_set.ipynb): Graphillionではグラフの集合を表すGraphSetとよばれるオブジェクトを作成し，様々な操作を実行することができます．この章ではGraphSetに対して行える代表的な操作を紹介します．

7. [Graphillionによる最適化](07_optimization.ipynb): Graphillionを用いることで，グラフに関する最適化問題を解くことができます．この章ではGraphillionを用いて最適化問題を解く方法と，Graphillionが得意とする最適化問題の種類について説明します．
8. [ネットワーク信頼性の計算](08_network_reliability.ipynb): Graphillionの機能を活用することで，ネットワークの故障に対する強さを調べることができます．
9. [Graphillion 実践ガイド](08_practtical_guide.ipynb) Graphillionで効率的に問題を解くための指針をいくつか紹介します．